In [305]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pysr import PySRRegressor
from IPython.display import Markdown as md

This file pretends to fit data from CLASS to obtain an analytic formula for T(k). Now let's import T(k):

def accuracy(data, model):
    if len(data) != len(model):
        print('ERROR: array must have the same length!')
        return
    
    l = len(data)
    b = 0
    
    for i in range(l):
        a = abs((data[i] - model[i]) / data[i])
        b += a
    
    return 100 * b / l

In [306]:
def error(data,model):
    if len(data) != len(model):
        print('ERROR: array must have the same length!')
        return
    
    l = len(data)
    error = []
    
    for i in range(l):
        error.append(abs((data[i] - model[i])**2 / data[i]**2))
    
    return np.array(error)

In [307]:
def accuracy(data, model):
    if len(data) != len(model):
        print('ERROR: array must have the same length!')
        return
    
    l = len(data)
    total_error = 0
    
    for i in range(l):
        error = abs((data[i] - model[i]) / data[i])
        total_error += error
    
    avg_error = total_error / l
    accuracy_percentage = 100 * (1 - avg_error)
    
    return accuracy_percentage


In [308]:
tf = pd.read_csv('TF_16.csv')
k = tf['k (h/Mpc)'].values
x = tf['x'].values
T = tf['T(k)'].values
x1 = x

Let's define a Regressor:

In [ ]:
model = PySRRegressor(
    model_selection="best",
    niterations=1000,  # < Increase me for better results
    
    binary_operators=["+","*","-","/","^"],
    constraints={'^': (2, 1)},
    nested_constraints={"^": {"^": 2}},
    maxsize=20,
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
    turbo=True,  
    #cluster_manager=cluster[1],
    #multithreading=True,
)

In [ ]:
model.from_file('96accuracy-2908.pkl')

In [ ]:
df = pd.read_csv('96accuracy-2908.csv')
print(df['Equation'][10])

try #2 (hall_of_fame_2023-08-29_163723.264.csv)

$$    T(x)=\frac{{(0.6802852^{0.07809298})}}{{x_0 + \left(\left(\frac{{(x_0^{1.5150192})}}{{0.07809298}} + x_0\right) + x_0\right) \cdot (x_0 \cdot 0.6802852)}} \cdot x_0 = \frac{1}{1.031+1.402x+8.977x^{1.515}}$$

In [ ]:
T_f = (((0.6802852**0.07809298) / (x + (((((x**1.5150192) / 0.07809298) + x) + x) * (x * 0.6802852)))) * x)
pr = 1 / (1.031 + 8.977 * x ** 1.515 + 1.402 * x)

In [ ]:
plt.figure()
plt.scatter(x,T, s=1, label='CLASS data')
#plt.scatter(x, T_f, s=1, label='From PySR')
plt.scatter(x, pr, s=1, label='From PySR')
plt.loglog()
plt.legend()
plt.show()


In [ ]:
accuracy(T,T_f)

In [ ]:
plt.scatter(k,error(T,T_f))

try #3
$$\frac{0.974}{x_{0} + 8.67 x_{0}^{1.44} + 1.00}$$

In [ ]:
T_2 = 0.974 / (x + 8.67 * x ** 1.44 + 1.00)

In [ ]:
plt.scatter(x,T, s=1, label='CLASS data')
plt.scatter(x, T_2, s=1, label='From PySR')
plt.loglog()
plt.legend()
plt.show()

In [ ]:
accuracy(T,T_2)

In [ ]:
plt.scatter(k,error(T,T_2))